## **Classification Challenge**

Wine experts can identify wines from specific vineyards through smell and taste, but the factors that give different wines their individual charateristics are actually based on their chemical composition.

In this challenge, you must train a classification model to analyze the chemical and visual features of wine samples and classify them based on their cultivar (grape variety).

> **Citation**: The data used in this exercise was originally collected by Forina, M. et al.
>
> PARVUS - An Extendible Package for Data Exploration, Classification and Correlation. Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno, 16147 Genoa, Italy.
>
> It can be downloaded from the UCI dataset repository (Dua, D. and Graff, C. (2019). UCI Machine Learning Repository. Irvine, CA: University of California, School of Information and Computer Science).

### **Explore the data**

Let's hit the ground running by importing a CSV file of wine data, which consists of 12 numeric features and a classification label with the following classes:

-   **0** (*variety A*)

-   **1** (*variety B*)

-   **2** (*variety C*)


In [ ]:
# Load the required packages into your current R session
suppressPackageStartupMessages({
  library(tidyverse)
  library(tidymodels)
  library(here)
  library(janitor)
})

# Read the csv file into a tibble
data <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/challenges/data/wine.csv", show_col_types = FALSE)

# Print the first 10 rows of the data
data %>% 
  slice_head(n = 10)


Your challenge is to explore the data and train a classification model that achieves an overall *Recall* metric of over 0.95 (95%).

Let's kick off our adventure by reformatting the data to make it easier for a model to use effectively.


In [ ]:
# Load the janitor package for cleaning data
library(janitor)

# Clean data a bit
wine_data <- data %>%
  # Encode WineVariety as category
  mutate(WineVariety = factor(WineVariety, levels = c("0","1", "2"))) %>% 
  # Clean column names
  clean_names()


# View data set
wine_data %>% 
  glimpse()


The goal of data exploration is to try to understand the `relationships` between its attributes; in particular, any apparent correlation between the *features* and the *label* your model will try to predict. One way of doing this is by using data visualization.

Now let's compare the feature distributions for each label value.


In [ ]:
theme_set(theme_light())
# Pivot data to a long format
wine_data %>% 
    pivot_longer(!wine_variety, names_to = "features", values_to = "values") %>% 
# Make a box plot for each predictor feature
  ggplot(mapping = aes(x = wine_variety, y = values, fill = features)) +
  geom_boxplot() + 
  facet_wrap(~ features, scales = "free", ncol = 5) +
  scale_color_viridis_d(option = "plasma", end = .7) +
  theme(legend.position = "none")


What insights about the wine varieties have you derived from the distribution of the different features?

### Split the data for training and validation

It is best practice to hold out some of your data for **testing** in order to get a better estimate of how your models will perform on new data by comparing the predicted labels with the already known labels in the test set.


In [ ]:
# Load the Tidymodels packages
library(tidymodels)



# Split data into 70% for training and 30% for testing
set.seed(2056)
wine_split <- wine_data %>% 
  initial_split(prop = 0.70)


# Extract the data in each split
wine_train <- training(wine_split)
wine_test <- testing(wine_split)


# Print the number of cases in each split
cat("Training cases: ", nrow(wine_train), "\n",
    "Test cases: ", nrow(wine_test), sep = "")


### Data preprocessing with recipes

Now that we have a set of training features and corresponding training labels, we can fit a multiclass classification algorithm to the data to create a model.

`parsnip::multinom_reg()` defines a model that uses linear predictors to predict multiclass data using the multinomial distribution.


In [ ]:
# Specify a multinomial regression via nnet
mr_spec <- multinom_reg(
  penalty = 1,
  engine = "nnet",
  mode = "classification"
)


# Create a recipe that specifies that predictors should be on the same scale
mr_wflow <- recipe(wine_variety ~ ., data = wine_train) %>% 
  step_normalize(all_numeric_predictors()) %>% 
# Bundle recipe and model specification into a workflow
  workflow(mr_spec)

# Print out workflow
mr_wflow


After a workflow has been *specified*, a model can be `trained` using the [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) function.



In [ ]:
# Fit a workflow object
mr_wflow_fit <- mr_wflow %>% 
  fit(data = wine_train)

# Print wf object
mr_wflow_fit


### Evaluate model performance

Good job👏! We now have a trained workflow. The workflow print out shows the coefficients learned during training.

This allows us to use the model trained by this workflow to predict labels for our test set and evaluate performance. Let's obtain the hard class and probability predictions for our test set.


In [ ]:
# Make predictions on the test set
results <- mr_wflow_fit %>% 
  augment(wine_test) 

# Print the results
results %>% 
  slice_head(n = 10)


Now, let's look at the confusion matrix for our model



In [ ]:
# Confusion matrix
results %>% 
  conf_mat(wine_variety, .pred_class)


The confusion matrix shows the intersection of predicted and actual label values for each class - in simple terms, the diagonal intersections from top-left to bottom-right indicate the number of correct predictions.

When dealing with multiple classes, it's generally more intuitive to visualize this as a heat map, like this:


In [ ]:
update_geom_defaults(geom = "tile", new = list(color = "black", alpha = 0.7))
# Visualize confusion matrix
results %>% 
  conf_mat(wine_variety, .pred_class) %>% 
  autoplot(type = "heatmap")


The darker squares in the confusion matrix plot indicate high numbers of cases, and you can hopefully see a diagonal line of darker squares indicating cases where the predicted and actual label are the same.

Let's now calculate summary statistics for the confusion matrix.


In [ ]:
# Statistical summaries for the confusion matrix
conf_mat(data = results, truth = wine_variety, estimate = .pred_class) %>% 
  summary() %>% 
  filter(.metric %in% c("accuracy", "recall", "precision"))


The tibble shows the overall metrics of how well the model performs across all three classes of wines. Not bad!

Let's now evaluate the ROC metrics. In the case of a multiclass classification model, a single ROC curve showing true positive rate vs false positive rate is not possible. However, you can use the rates for each class in a One vs Rest (OVR) comparison to create a ROC chart for each class.


In [ ]:
# Make a ROC_CURVE
results %>% 
  roc_curve(wine_variety, c(.pred_0, .pred_1, .pred_2)) %>% 
  ggplot(aes(x = 1 - specificity, y = sensitivity, color = .level)) +
  geom_abline(lty = 2, color = "gray80", size = 0.9) +
  geom_path(show.legend = T, alpha = 0.6, size = 1.2) +
  coord_equal()


To quantify the ROC performance, you can calculate an aggregate area under the curve score that is averaged across all of the OVR curves.



In [ ]:
# Calculate ROC_AOC
results %>% 
  roc_auc(wine_variety, c(.pred_0, .pred_1, .pred_2))


That went down well! Overall, the model did a great job in classifying the wine varieties. Be sure to try out a different classification algorithm.

### **Use the model with new data observation**

When you're happy with your model's predictive performance, save it and then use it to predict classes for the following two new wine samples:

-   [13.72, 1.43, 2.5, 16.7, 108, 3.4, 3.67, 0.19, 2.04, 6.8, 0.89, 2.87, 1285]

-   [12.37, 0.94, 1.36, 10.6, 88, 1.98, 0.57, 0.28, 0.42, 1.95, 1.05, 1.82, 520]


In [ ]:
library(here)
# Save trained workflow
saveRDS(mr_wflow_fit, "wine_mr_model.rds")


Now, we can load it whenever we need it, and use it to predict labels for new data. This is often called *`scoring`* or *`inferencing`*.



In [ ]:
# Create a tibble for the new wine samples
new_wines <- c(13.72, 1.43, 2.5, 16.7, 108, 3.4, 3.67, 0.19, 2.04, 6.8, 0.89, 2.87, 1285) %>% 
  rbind(c(12.37, 0.94, 1.36, 10.6, 88, 1.98, 0.57, 0.28, 0.42, 1.95, 1.05, 1.82, 520)) %>% 
  as_tibble()
names(new_wines) = as.character(names(wine_data)[1:13])

new_wines


In [ ]:
# Load the model into the current R session
loaded_model <- readRDS("wine_mr_model.rds")

# Make predictions
predictions <- loaded_model %>% 
  augment(new_data = new_wines)

predictions


Happy Learning,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.
